In [1]:
import math
import functools

from utils.text_deal import get_final_split_result,get_doc_list

from utils.feature_extraction import train_idf,TfIdf

In [4]:
with open("./data/FT_Camp_3/demo.txt",'r') as file_object:
    text = file_object.read()
dealed_texts = get_final_split_result(text)

doc_list = get_doc_list(dealed_texts)
idf_dic = train_idf(doc_list)

words = sorted(list(idf_dic))

tfidf = TfIdf(idf_dic,1,doc_list[1],words=words)

tfidf_vec = []
for i,doc in enumerate(doc_list):
    tfidf = TfIdf(idf_dic,1,doc,words)
    tfidf_vec.append(tfidf.get_tfidf())



In [5]:
len(tfidf_vec)

14

In [9]:

class DBSCAN():
    def __init__(self,epsilon,MinPts):
        self.epsilon=epsilon
        self.MinPts=MinPts
        ###距离矩阵
        self.dist=0
        ###所有簇集合
        self.k_clusters=[]
        ###核心点
        self.CorePts=np.array([],dtype=int)
        ###当前样本
        self.Samples=0
        ###当前簇
        self.clusters=0
    
    ###用来一直往下找核心点
    def findDensity(self,point):
        ###将该核心点从核心点集合去掉
        self.CorePts=np.setdiff1d(self.CorePts,point)
        ###找到该核心点的密度直达点，与样本取交集，即防止点被重复聚类
        densityPts=np.where((self.dist[int(point)]<self.epsilon)==True)[0]
        densityPts=np.intersect1d(densityPts,self.Samples)
        ###找到密度直达点中的核心点
        IntersecCorePts=np.intersect1d(self.CorePts,densityPts)
        ###将这些点添加进目前的簇
        self.clusters=np.append(self.clusters,densityPts)
        self.clusters=np.unique(self.clusters)
        ###将这些点从样本中移除
        self.Samples=np.setdiff1d(self.Samples,self.clusters)
        ###从该点邻域的核心点出发，只要还能找得到核心点，就一直往下找
        if len(IntersecCorePts)!=0:
            for IntersecCore in IntersecCorePts:
                self.findDensity(IntersecCore)
        return self.clusters
    
    def fit(self,data):
        m=data.shape[0]
        self.dist=np.zeros((m,m))
        self.Samples=np.arange(m)
        for datum,idx in zip(data,range(m)):
            self.dist[idx]=np.sqrt(np.sum(np.square(datum-data),axis=1))
            ###加入核心点
            if (np.sum(self.dist[idx]<=self.epsilon)) >= self.MinPts:
                self.CorePts=np.append(self.CorePts,idx)
        ###只要核心点集合不为空，一直找
        while(len(self.CorePts)!=0):
            self.clusters=np.array([],dtype=int)
            c=self.findDensity(np.random.choice(self.CorePts))
            self.k_clusters.append(c)
        return self.k_clusters  


In [11]:
model=DBSCAN(0.11,2)

NameError: name 'np' is not defined

In [14]:
import sys, math, random, csv, types

class KMeans:
    def __init__(self,k):
        self.data, self.restrictions = self.parse_vectors(filename)
        self.k = k
        self.centroids = self.initial_centroids(self.data, self.k)
        self.clusters = list()

    

    def initial_centroids(self, data, k):
        # initialize c (initial centroid) to 0
        row_len = len(data[0]['val'])
        c = [0 for i in range(row_len)]

        for point in data:
            for ndx, val in enumerate(c):
                c[ndx] += float(point['val'][ndx])

        for i in range(row_len):
            c[i] = c[i]/len(data)

        centroids = []
        
        for m in range(int(k)):
            maxdist = 0
            maxndx = 0
            for ndx, point in enumerate(data):
                if len(centroids) > 0:
                    if not any(d.get('val') == point['val'] for d in centroids):
                        dist = self.distance(point['val'], centroids[m-1]['val'])
                else:
                    dist = self.distance(point['val'], c)

                if maxdist < dist:
                    maxdist = dist
                    maxndx = ndx
                    
            cluster = dict()
            cluster['cluster'] = m
            cluster['val'] = data[maxndx]['val']
            centroids.append(cluster)

        return centroids

    def cluster(self,input_data):
        change = True

        while change:
            change = False

            for i, point in enumerate(self.data):
                # find the closest centroid
                closest_cluster = self.closest(point)
                if closest_cluster != point['cluster']:
                    change = True
                    self.data[i]['cluster'] = closest_cluster
            
            if change:
                self.recalc_centroids()

        for centroid in self.centroids:
            if 'key' in self.data[0]:
                cluster_points = ([(x['key'], x['val']) for x in self.data if x['cluster'] == centroid['cluster']])
            else:
                cluster_points = ([x['val'] for x in self.data if x['cluster'] == centroid['cluster']])
            self.clusters.append(cluster_points)

        return self.clusters

    def recalc_centroids(self):
        row_len = len(self.data[0]['val'])
        c = [[0 for i in range(row_len)] for j in range(self.k)]
        for point in self.data:
            cluster_ndx = point['cluster']
            for ndx, val in enumerate(c[cluster_ndx]):
                c[cluster_ndx][ndx] += float(point['val'][ndx])

        for cluster_ndx, x in enumerate(c):
            cluster_count = len([d for d in self.data if d['cluster']==cluster_ndx])
            if cluster_count != 0:
                for i in range(row_len):
                    c[cluster_ndx][i] = c[cluster_ndx][i]/cluster_count
        
        for i, centroid in enumerate(self.centroids):
            self.centroids[i]['val'] = c[i]

    def distance(self, v1, v2):
        if len(v1) != len(v2): 
            print('Error can not compute distance between unequal vector lengths.')
            return
        else:
            total = 0
            for i in range(len(v1)):
                val = float(v1[i]) - float(v2[i])
                val = val * val
                total += val
            return math.sqrt(total)

    def closest(self, point):
        minDist = sys.maxint
        minCluster = -1
        for ndx, cluster in enumerate(self.centroids):
            dist = self.distance(cluster['val'], point['val'])
            if minDist > dist:
                minDist = dist
                minCluster = cluster['cluster']

        return minCluster

def main():
    filename = 0
    threshold = 0

    if len(sys.argv) > 3 or len(sys.argv) < 2:
        print("Usage: python kmeans.py <filename> num_clusters")
        return    
    
    if len(sys.argv) == 3:
        numClusters = int(sys.argv[2])

    kmeans = KMeans(sys.argv[1], numClusters)
    clusters = kmeans.cluster()

    for i, cluster in enumerate(clusters):
        print("Cluster "+str(i+1)+":")
        centroid = kmeans.centroids[i]['val']
        print("Center: ", centroid)
        print("Num Points: ", len(cluster))
        print("Points: ")
        maxdist = 0
        mindist = sys.maxint
        avgdist = 0
        for point in cluster:
            if type(point[1]) == tuple: 
                distance = kmeans.distance(centroid, point[1])
            else:
                distance = kmeans.distance(centroid, point)

            print(point, ", distance=", distance)
            avgdist += distance
            if distance > maxdist:
                maxdist = distance
            if distance < mindist:
                mindist = distance
        
        print("Max distance: ", maxdist)
        print("Min distance: ", mindist)
        print("Avg distance: ", avgdist/len(cluster), "\n")
            

if __name__ == '__main__':
    main()

ValueError: invalid literal for int() with base 10: '/run/user/1009/jupyter/kernel-b36371da-b94b-45f4-a8b6-a02a38948fad.json'